<a href="https://colab.research.google.com/github/apetros/pyeplan/blob/master/examples/1_bus_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a microgrid considering all generation and load connected to single bus

This simple example neglects the network model and considers a single node with total load. One representative day with is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, wind units, solar units and diesel generation units. It showcases how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the microgrid network.

This example is designed to be run on Google Colab.

In [1]:
!rm -r sample_data
!rm -r 1_bus

rm: cannot remove '1_bus': No such file or directory


In [2]:
!apt-get install -qq subversion
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/1_bus

Selecting previously unselected package libapr1:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libapr1_1.6.3-2_amd64.deb ...
Unpacking libapr1:amd64 (1.6.3-2) ...
Selecting previously unselected package libaprutil1:amd64.
Preparing to unpack .../libaprutil1_1.6.1-2_amd64.deb ...
Unpacking libaprutil1:amd64 (1.6.1-2) ...
Selecting previously unselected package libserf-1-1:amd64.
Preparing to unpack .../libserf-1-1_1.3.9-6_amd64.deb ...
Unpacking libserf-1-1:amd64 (1.3.9-6) ...
Selecting previously unselected package libsvn1:amd64.
Preparing to unpack .../libsvn1_1.9.7-4ubuntu1_amd64.deb ...
Unpacking libsvn1:amd64 (1.9.7-4ubuntu1) ...
Selecting previously unselected package subversion.
Preparing to unpack .../subversion_1.9.7-4ubuntu1_amd64.deb ...
Unpacking subversion (1.9.7-4ubuntu1) ...
Setting up libapr1:amd64 (1.6.3-2) ...
Setting up libaprutil1:amd64 (1.6.1-2) ...
Setting up libserf-1-1:amd64 (1.3.9-6) ...
Setting up libsvn1

In [3]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155359 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [4]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155562 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [5]:
!pip install -q pyeplan

     |████████████████████████████████| 43.0 MB 1.8 MB/s 
     |████████████████████████████████| 23.1 MB 1.4 MB/s 
     |████████████████████████████████| 11.5 MB 64.3 MB/s 
     |████████████████████████████████| 9.2 MB 21.6 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [6]:
!pip install git+git://github.com/SPS-L/pyeplan@master

  Cloning git://github.com/apetros/pyeplan (to revision master) to /tmp/pip-req-build-_h5zxhby
  Running command git clone -q git://github.com/apetros/pyeplan /tmp/pip-req-build-_h5zxhby
  Created wheel for pyeplan: filename=pyeplan-0.4.3-py3-none-any.whl size=16411 sha256=c126e65edd9ab5683b3da9ec95e61961db88dfd592581d73ffedc6c09f21e93f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5caxsywu/wheels/9e/05/40/2374125e8f55142a752f54005fbe165970a6dda55bb008fbf0
Successfully built pyeplan
  Attempting uninstall: pyeplan
    Found existing installation: pyeplan 0.4.2
    Uninstalling pyeplan-0.4.2:
      Successfully uninstalled pyeplan-0.4.2


## Defining the input data

The next step is to define the directory to the folder where the input data is located and define the reference node. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).

Some of the data inputs include:

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [7]:
import pandas as pd
pd.read_csv("1_bus/prep_dist.csv")

,0
0,90


### Defining the input investment candidates

#### Wind Units

In [8]:
pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


#### Solar PV Units

In [9]:
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


#### Diesel/Fossil Units

In [10]:
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,0,0,0,NaN
1,0,12,0.4,0,0,0,0,0,NaN


## Case studies
Case 1: No diesel candidates

Case 2: Solar + Wind + diesel candidated

Case 3: Total load reduced from 90 kW to 70 kW

## Using the invesment and operation planning module from PyEPLAN 

In [11]:
import pyeplan as pye

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [12]:
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = nominal. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= nominal.

In [13]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = True, )

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpvmngxif3.glpk.raw --wglp /tmp/tmpo21pfrkn.glpk.glp --cpxlp
 /tmp/tmppb_8nk6_.pyomo.lp
Reading problem data from '/tmp/tmppb_8nk6_.pyomo.lp'...
/tmp/tmppb_8nk6_.pyomo.lp:248: warning: lower bound of variable 'xg(0)' redefined
/tmp/tmppb_8nk6_.pyomo.lp:248: warning: upper bound of variable 'xg(0)' redefined
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
254 lines were read
Writing problem data to '/tmp/tmpo21pfrkn.glpk.glp'...
217 lines were written
GLPK Integer Optimizer, v4.65
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
Preprocessing...
9 rows, 14 columns, 28 non-zeros
6 integer variables, all of which are binary
Scaling...
 A: min|aij| =  3.333e-01  max|aij| =  4.000e+01  ratio =  1.200e+02
GM: min|aij| =  7.598e-01  max|aij| =  1.316e+00  ratio =  1.732e+00
EQ: min|aij| =  5.774e-01  max|aij| =  1.000e+00  ratio =  1.732e+00

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total Investment and Operational Costs

In [14]:
inosys.resCost()

,total costs,20200.0
0,total investment costs,20200.0
1,total operation costs,0.0


### Number and capacity of wind units installed

The capacity and location of wind units installed is:

In [15]:
inosys.resWind()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,0.0,0


### Number and capacity of solar units installed

The capacity and location of solar units installed is:

In [16]:
inosys.resSolar()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,50.0,0


### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [17]:
inosys.resConv()

,Installed Capacity (kW),Bus
Unit,,
1,0.0,0
2,0.0,0


### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [18]:
inosys.resCurt()

,0
Hour,
0,1.184238e-16
